In [1]:
from vision import Vision
from globalPlanning import GlobalPlanning
from localPlanning import LocalPlanning
from kalman import Kalman
from plotter import Plotter
from thymioControl import ThymioControl

import cv2
import math
import sys
import time
import numpy as np


In [2]:
import tdmclient.notebook
await tdmclient.notebook.start()

In [3]:
@tdmclient.notebook.sync_var
def motor_go(left,right):
    global motor_left_target, motor_right_target
    motor_left_target = left
    motor_right_target = right
    
@tdmclient.notebook.sync_var
def motor_stop():
    global motor_left_target,motor_right_target
    motor_left_target = 0
    motor_right_target = 0
    
@tdmclient.notebook.sync_var
def sensor_data():
    global prox_horizontal
    return prox_horizontal.copy()

@tdmclient.notebook.sync_var
def leds():
    global leds_top, leds_bottom_left, leds_bottom_right, leds_temperature, leds_circle
    leds_top = [0, 0, 0]
    leds_bottom_left = [0, 0, 0]
    leds_bottom_right = [0, 0, 0]
    leds_temperature = [0, 0]
    leds_circle = [0, 0, 0, 0, 0, 0, 0, 0]

VISION

In [4]:

# if __name__ == "__main__":
#     try:
#         vision = Vision(target_height=20, fps=3, threshold=128)

#         while True:
#             vision.update()
#             vision.display_matrix()
#             vision.display_info()

#             if cv2.waitKey(1) & 0xFF == ord('q'):
#                 break
#     finally:
#         vision.release()


GLOBAL PLANNING

The goal of this project is to achieve a position with the robot. To make this happen, we will use the global navigation. It's the same principle than follow a GPS, we build a path that our thymio will follow. This path start from the position of the thymio, and ends at the position of our goal placed on our environment.

In [5]:
# # Main funcion to test the GlobalPlanning class
# import numpy as np
# globalPlanning = GlobalPlanning()
# matrix = np.zeros((15, 15))
# matrix[1:6, 6:9] = -1; matrix[11, 1] = -1
# start = (1, 1)
# goal = (10, 10)
# if start == None:
#     print("No start found")
# elif goal == None:
#     print("No goal found")
# else:
#     #path, matrix2 = globalPlanning.dijkstra(vision.get_matrix(), vision.get_start(), vision.get_goal)
#     path, matrix2 = globalPlanning.dijkstra(matrix, start, goal)
#     print(path)
#     print(matrix2)
#     plotter = Plotter()
#     plotter.plot_map_given(matrix2, start, goal)

LOCAL PLANNING

Once the global navigation module has designed a path from the start position to the goal, the local planning module has to generate the movement commands for the robot in such a way that it will follow that path.

An important part in robot's navigation is the avoidance of unexpected obstacles that might be detecte while moving from the start position to the goal, following the global plan. The process of local planning consists of detecting the obstacles using the robots sensors and designing a more or less efficient plan to get around it, avoiding collisions, and get back on the predetermined global plan.

The Thymio robot features 5 horizontal proximity sensors it its front part (see Thyimio cheat sheet snippet) that can be used to detect obstacles using infrared technology. The range of values that the sensors return is [0, 4300], ranging from nothing detected to something detected at minimum distance from the sensor, and the updates come at a frequency of 10Hz.
![image.png](images/thymio_cheat_sheet1.png)

Inputs: horizontal proximity sensors values
Outputs: translation and rotation values for the robot

In [6]:
# Parameters
threshold = 1000

: 

FLITERING

### The Role of the Kalman Filter in the Project

The Kalman Filter is a mathematical tool used in this project to estimate the robot's position and orientation (state) while navigating toward a goal. The robot uses data from two sources: **odometry** (wheel speeds) for prediction and a **camera** for occasional updates. Since both these sources are noisy and prone to errors, the Kalman Filter combines their information to produce an accurate and smooth estimate of the robot’s state.


#### **Nonlinear Motion Model**
A differential-drive robot's motion is inherently nonlinear because its position $(x, y)$ and orientation $\theta$ depend on trigonometric relationships. For example, its state evolves as (motion model):

$$
x_{t+1} = x_t + v \cos(\theta) \Delta t, \quad y_{t+1} = y_t + v \sin(\theta) \Delta t, \quad \theta_{t+1} = \theta_t + \omega \Delta t
$$

Here:
- $v$: Linear velocity (derived from wheel speeds).
- $\omega$: Angular velocity (based on differential wheel motion).
- $\Delta t$: Time step.

The EKF linearizes these equations using the Jacobian of the motion model. In the **Kalman class**, the motion model is embedded in the matrix $\mathbf{G}$

### Fusing Odometry and Camera Data

Odometry (wheel encoder data) is prone to errors such as:

- **Drift** over time, accumulating inaccuracies as the robot moves.
- **Wheel slippage** and uneven terrain causing deviations from the true trajectory.

The camera, while more accurate, provides intermittent data due to potential obstructions or missed detections. The EKF combines these two sources of data to provide a reliable state estimate:

1. **Prediction Step**: This step uses the odometry data to predict the robot's next state. It always runs, regardless of whether camera data is available, ensuring continuity in state estimation. The prediction compensates for missing measurements and accounts for process noise.

2. **Update Step**: This step completes the prediction using camera measurements when available. It refines the estimated state by fusing the relatively accurate camera data with the prediction. The update step only runs if the camera is not obstructed.

### **3. How the Kalman Filter Works**

#### **A. Prediction Step**

Using the robot's last known state, wheel speeds, and time elapsed (Δt), the Kalman Filter predicts the robot's next state:

$$
\mathbf{E}_{\text{pred}} = \mathbf{A} \cdot \mathbf{E} + \mathbf{G} \cdot \mathbf{U}
$$

- **A**: State transition matrix (accounts for constant movement without changes).
- **B**: Control matrix (maps wheel speeds to motion).
- **U**: Control vector (wheel speeds).

It also predicts the **uncertainty** in the state, denoted by the covariance matrix **P**:

$$
\mathbf{P}_{\text{pred}} = \mathbf{A} \cdot \mathbf{P} \cdot \mathbf{A}^T + \mathbf{R}
$$

- **Q**: Process noise covariance (uncertainty from wheel encoders).

#### **B. Update Step**

When the camera provides a measurement **Z**, the Kalman Filter updates the state and uncertainty:

1. Compute the Kalman gain **K**, which determines how much weight to give to the measurement:

$$
\mathbf{K} = \mathbf{P}_{\text{pred}} \cdot \mathbf{H}^T \cdot \left(\mathbf{H} \cdot \mathbf{P}_{\text{pred}} \cdot \mathbf{H}^T + \mathbf{Q}\right)^{-1}
$$

- **H**: Measurement matrix (maps state to camera readings).
- **R**: Measurement noise covariance (uncertainty in camera data).

2. Update the state using the measurement:

$$
\mathbf{E}_{\text{update}} = \mathbf{E}_{\text{pred}} + \mathbf{K} \cdot \left(\mathbf{Z} - \mathbf{H} \cdot \mathbf{E}_{\text{pred}}\right)
$$

3. Update the uncertainty:

$$
\mathbf{P}_{\text{update}} = (\mathbf{I} - \mathbf{K} \cdot \mathbf{H}) \cdot \mathbf{P}_{\text{pred}}
$$


### **Behavior in the Control Loop**

1. **Prediction Always Happens**:
    * The prediction step runs at every iteration using odometry.
    * This ensures the robot always has an estimated position, even if the camera data is unavailable.

2. **Update Happens When the Camera Sees the Robot**:
    * If the camera detects the robot, the Kalman Filter corrects its prediction using the more accurate camera measurement.

3. **Handling Obstructions**:
    * If the camera is obstructed, the filter relies solely on the prediction step until the camera resumes detection.


MOTION CONTROL

CONTROL LOOP

In [ ]:
x_trajectory = []
y_trajectory = []
kalman_positions = []

# initialize objects
globalPlanning = GlobalPlanning()
localPlanning = LocalPlanning()
filter = Kalman()
thymio = ThymioControl()
plotter = Plotter()

dt = 0.15

# create a Vision object
image_path3 = "images/IMG_7028.jpeg"
vision = Vision(fps=3,target_height=200, default_image_path=image_path3)

iter = 1

goal_pos = [0, 0]

goal = False

leds()

while not goal:
    vision.update_image(live=False)
    position = vision.getStart()
    angle = vision.getAngle()

    if (iter == 1):
        print("First iteration")
        print("Getting map and Thymio position from camera")
        vision.update_image()
        map = vision.getMatrix()
        thymio.set_pose(position.copy(), angle)

        if (position is None):
            print("Error: camera obstructed in the first iteration")
            exit()
                  
        print("Computing path")
        print("Map shape: ", map.shape)
        goal_pos = vision.getGoal()
        path = globalPlanning.dijkstra(map, position, goal_pos)
        thymio.set_path(path)
        plotter.set_map(map, position, goal_pos)
        plotter.plot_path(thymio.get_path_cells())
        print("Path: ", thymio.get_path_cells())
        x_trajectory.append(position[0])
        y_trajectory.append(position[1])
        filter.initialize_position(thymio.get_position()[0], thymio.get_position()[1], angle)
        filter.set_lastKalman_time()

    # check if camera is obstructed
    if position is not None:
        thymio.update_pose(position, angle)
        print("Camera not obstructed, getting position from camera")
        x_trajectory.append(float(position[0]))
        y_trajectory.append(float(position[1]))
        # camera measuerement that will then be used for the Kalman filter
        measurement = np.array([thymio.get_position()[0], thymio.get_position()[1], angle])
        filter.kalman_update(measurement)
        print("Kalman update: ", filter.get_state())
        
        # check if the robot has been kidnapped
        if thymio.amIKidnapped():
            print("Kidnapping detected")

            # tmdclient function to stop the motors
            motor_stop()

            # update the map with new robot position
            time.sleep(2)
            vision.update_image()
            map = vision.getMatrix()
            position = vision.getStart()
            angle = vision.getAngle()

            # new path planning
            path = globalPlanning.dijkstra(map, position, goal_pos)
            thymio.set_path(path)
            plotter.plot_path(thymio.get_path_cells())
            x_trajectory.append(position[0])
            y_trajectory.append(position[1])
            filter.initialize_position(thymio.get_position()[0], thymio.get_position()[1], angle)
            filter.set_lastKalman_time()

        if vision.getGoal() != goal_pos:
            print("Goal changed")
            goal_pos = vision.getGoal()
            path = globalPlanning.dijkstra(map, position, goal_pos)
            thymio.set_path(path)
            plotter.plot_path(thymio.get_path_cells())
    
    else:
        print("Camera obstructed")

    # get the state from the Kalman filter
    # the result will depend on the mode of the filter
    x, y, angle = filter.get_state()
    kalman_position = [x, y]
    kalman_positions.append(position)
    print("Kalman position used: ", kalman_position)

    # check if the robot is detecting an obstacle
    # tmclient function to get the proximity sensors
    prox = sensor_data()
    if (localPlanning.is_local_planning(prox)):
        # move with local planning until the robot is not back on the path
        wl, wr = localPlanning.local_planning(prox)
        v, w = thymio.inverseDifferentialDrive(wl, wr)
    else:
        # move with global planning
        v, w, wl, wr, goal = thymio.move(kalman_position, angle)

    # update the Kalman filter
    filter.kalman_prediction(wl, wr)

    # tmdclient function to move the motors
    wl = int(wl)
    wr = int(wr)

    print("Kalman pos 0: ", kalman_position[0])
    print("Kalman pos 1: ", kalman_position[1])
    print("v, w ", v, w)
    print("angle: ", angle)
    print("New angle: ", (angle + w*dt) % (2 * math.pi))


    print("Trajectory: ", x_trajectory, y_trajectory)
    plotter.plot_trajectory(x_trajectory, y_trajectory)

    motor_go(wl, wr)
    
    # sleep for a while
    iter += 1
    time.sleep(dt)